<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 16.0 MB/s 
     |████████████████████████████████| 166 kB 63.9 MB/s 
     |████████████████████████████████| 182 kB 65.5 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 72.4 MB/s 
     |████████████████████████████████| 162 kB 70.1 MB/s 
     |████████████████████████████████| 162 kB 63.1 MB/s 
     |████████████████████████████████| 158 kB 72.5 MB/s 
     |████████████████████████████████| 157 kB 71.0 MB/s 
     |████████████████████████████████| 157 kB 69.3 MB/s 
     |████████████████████████████████| 157 kB 65.2 MB/s 
     |████████████████████████████████| 157 kB 75.7 MB/s 
     |████████████████████████████████| 157 kB 73.5 MB/s 
     |████████████████████████████████| 157 kB 26.0 MB/s 
     |████████████████████████████████| 157 kB 70.7 MB/s 
     |████████████████████████████████| 156 kB 59.6 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [ ]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [ ]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [ ]:
densenet121 = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)

model_densenet121=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  densenet121,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
densenet121.trainable=False
model_densenet121.summary()

33188688/33188688 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 1000)              8062504   
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 5)                 5005      
                                                                 
Total params: 8,067,509
Trainable params: 5,005
Non-trainable params: 8,062,504
_________________________________________________________________


In [ ]:
model_densenet121.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [ ]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "densenet121",
)

In [ ]:
%%time
epochs=40
history = model_densenet121.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.5990 - accuracy: 0.4716

wandb: Adding directory to artifact (/content/wandb/run-20221106_050017-jhmvizzn/files/model-best)... Done. 0.2s


22/22 [==============================] - 231s 7s/step - loss: 1.5990 - accuracy: 0.4716 - val_loss: 1.5884 - val_accuracy: 0.5657
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.5738 - accuracy: 0.6165

wandb: Adding directory to artifact (/content/wandb/run-20221106_050017-jhmvizzn/files/model-best)... Done. 0.2s


22/22 [==============================] - 50s 2s/step - loss: 1.5738 - accuracy: 0.6165 - val_loss: 1.5658 - val_accuracy: 0.6000
Epoch 3/40
22/22 [==============================] - ETA: 0s - loss: 1.5505 - accuracy: 0.6634

...

22/22 [==============================] - 50s 2s/step - loss: 1.1535 - accuracy: 0.6761 - val_loss: 1.1783 - val_accuracy: 0.6229
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 1.1491 - accuracy: 0.6761

wandb: Adding directory to artifact (/content/wandb/run-20221106_050017-jhmvizzn/files/model-best)... Done. 0.2s


22/22 [==============================] - 50s 2s/step - loss: 1.1491 - accuracy: 0.6761 - val_loss: 1.1723 - val_accuracy: 0.6229
Epoch 40/40
22/22 [==============================] - ETA: 0s - loss: 1.1399 - accuracy: 0.6719

wandb: Adding directory to artifact (/content/wandb/run-20221106_050017-jhmvizzn/files/model-best)... Done. 0.2s


22/22 [==============================] - 50s 2s/step - loss: 1.1399 - accuracy: 0.6719 - val_loss: 1.1666 - val_accuracy: 0.6286
CPU times: user 31min 36s, sys: 1min 2s, total: 32min 38s
Wall time: 42min 35s


In [ ]:
wandb.finish()

accuracy,▁▆██████▇▇▇▇█▇▇██▇██████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▄████▇▆▇▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇
val_loss,██▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
accuracy,0.67188
best_epoch,39
best_val_loss,1.16662
epoch,39
loss,1.13994
val_accuracy,0.62857


In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
